# SLR Parser
- ## Parse grammar
- ## first
- ## follow

In [167]:
import sys

G = {}
C = {}
start = ""
terminals = []
nonterminals = []
symbols = []
error=0
EPSILON = "/E"

grammars = open("grammar.txt")


In [107]:
from graphviz import Digraph


# Parse grammar

In [95]:
G = {}
start = ''
grammars = open("grammar.txt")
parse_grammar(grammars)


In [96]:
G
# ["L = R","R"]


{"S'": [['S']],
 'S': [['L', '=', 'R'], ['R']],
 'L': [['*', 'R'], ['id']],
 'R': [['L']]}

GRAMMAR:
 S -> L = R | R 
 L -> * R | id 
 R -> L 

AUGMENTED GRAMMAR:
0: S' -> S
1: S -> L = R
2: S -> R
3: L -> * R
4: L -> id
5: R -> L

TERMINALS    : ['=', '*', 'id']
NONTERMINALS : ["S'", 'S', 'L', 'R']
SYMBOLS      : ["S'", 'S', 'L', 'R', '=', '*', 'id']


In [102]:
def parse_grammar(grammars):
    """
       Grammar Parser
       args : grammars = file
       
    """
    global G, start, terminals, nonterminals, symbols
    for line in grammars: # read a single line from the grammar file
        line = " ".join(line.split()) # to remove \n
#         print(line)
        if line == '\n':
            break
        # head = line[:line.index("->")].strip()
        arrow_index = line.index("->") # arrow kata cha ta ?
        # print(arrow_index)

        # print(line[:arrow_index][1])
        head = line[:arrow_index].strip()  # Before Arrow ->
#         print(head)

        after_arrow = line[arrow_index + 2:]  # After Arrow ->
#         print(after_arrow)

        tails = after_arrow.split('|')  # Break each rule seperated by |
#         print(tails) 
        # ["   L = R    "," R  "]
#    "L = R" -> "L
        prods = [t.strip().split(' ') for t in tails]
        # Get single char from each tails given by the rule

#         print(prods)
        # now adding the production to its head in G 
#  {
#   "S'": [['S']],
#  'S': [['L', '=', 'R'], ['R']],
#  'L': [['*', 'R'], ['id']],
#  'R': [['L']]
# }
        if not start: 
            # For S' -> S
            start = head + "'"
            G[start] = [[head]]
            nonterminals.append(start)
            
        if head not in G:
            G[head] = []

        if head not in nonterminals:
            nonterminals.append(head)
        # L = []
        for prod in prods: # in  [['*', 'R'], ['id']]
            G[head].append(prod) # 1st time run huda -> ['*', 'R']
            # add the prod in non terminals / terminals
            for char in prod: # in  ['*', 'R']
                if char == EPSILON and char not in terminals:
                    terminals.append(char) # for epsilon
                # Check for small letters i.e terminals
                elif not char.isupper() and char not in terminals:
                    terminals.append(char) # for terimal

                # Check for capital letters i.e Non terminals
                # char not in terminals -> To stop Epsilon from getting append in Non terminoals
                elif char.isupper() and char not in nonterminals and char not in terminals:
                    nonterminals.append(char) 
                    G[char] = []    # non terminals dont produce other symbols
        # L = [['*', 'R'], ['id']]

    symbols =  nonterminals+terminals  # Merge all the notations/symbols

def print_grammar():
    print("GRAMMAR:")
    #  {
#   "S'": [['S']],
#  'S': [['L', '=', 'R'], ['R']],
#  'L': [['*', 'R'], ['id']],
#  'R': [['L']]
# }
    # Grammar without S', Given Grammar Cleaned
    # with | 
    for head in G.keys():
        if head == start:
            continue
        print(f" {head} -> ",end='') # S -> 
        num_prods = 0
        for prod in G[head]:
            if num_prods > 0:
                print("|",end=' ')
            print(f"{' '.join(prod)}",end=' ') # S -> L = R | R
            num_prods += 1
        print()

    print("\nAUGMENTED GRAMMAR:")
    i = 0
    for head in G.keys():
        for prod in G[head]:
            print(f"{str(i)}: {head} ->",end=' ')
            print(' '.join(prod)) # Join Single Production symbols with Space
            i += 1
    print()
    print(f"TERMINALS    : {terminals}")
    print(f"NONTERMINALS : {nonterminals}")
    print(f"SYMBOLS      : {symbols}")

AUGMENTED GRAMMAR:
0: S' -> S
1: S -> L = R
2: S -> R
3: L -> * R
4: L -> id
5: R -> L

In [169]:
parse_grammar(grammars)
print_grammar()

GRAMMAR:
 S -> L = R | R 
 L -> * R | id 
 R -> L 

AUGMENTED GRAMMAR:
0: S' -> S
1: S -> L = R
2: S -> R
3: L -> * R
4: L -> id
5: R -> L

TERMINALS    : ['=', '*', 'id']
NONTERMINALS : ["S'", 'S', 'L', 'R']
SYMBOLS      : ["S'", 'S', 'L', 'R', '=', '*', 'id']


# first

In [105]:
first_seen = []
def FIRST(X):
    """ 
        input : X = terminals,non terminals
        return : array of first of given X
    """
    global first_seen
    first = []
    # first_seen = []
    first_seen.append(X)
    if X in terminals:  # CASE 1
        first.append(X) # terminal So add

    elif X in nonterminals:  # CASE 2 Found Non terminal is 1st
        non_term_prods = G[X] # 
        for production in non_term_prods:
            # CASE 2.1, first of non terminal is terminal
            if production[0] in terminals and production[0] not in first:
                # NON terminals
                first.append(production[0])
            else:  # CASE 2.2, first of non terminal is non-terminal
                for_break = False
                for sym in production:
                # nonterm = production[0]
                    if for_break or sym in terminals:
                        break

                    if sym not in first_seen:
                        first_of_nonterm = FIRST(sym) # recursive call

                        for term in first_of_nonterm:
                            if term not in first:
                                first.append(term)

                        if EPSILON not in first_of_nonterm:
                            for_break = True
                                # FIRST(production)
    first_seen.remove(X)
    return first

def print_first():
    print("\nFIRST:")
    for head in G.keys(): # loop through each nonterminals in G
        print(f"{head} = {{",end=' ')
        terms = FIRST(head) # find the first of a single head/non-terminals

        print(' , '.join(terms),end=' ')
        print("}")

In [168]:
print_first()


FIRST:


# follow

In [164]:
follow_seen = []
def FOLLOW(passed_head):
    global follow_seen
    follow = []
    follow_seen.append(passed_head)

    # CASE 1
    if passed_head == start: # S'
        """
        follow of start symbol is $
        """
        follow.append('$')
        follow_seen.remove(passed_head)
        return follow

    for heads in G.keys():
        for prods in G[heads]:
            follow_head = False # head ko follow nikalne ki na nikalne
            if passed_head in prods: # right side of arrow contains passed_head or not
                next_symbol_pos = prods.index(passed_head) + 1 # passed_head front sym
                # CASE 2 net_symbol_pos not is last 
                while next_symbol_pos < len(prods):
                    """
                     Not in last postion, FIRST works for both terminal and
                     non terminal
                    """
                    symbol_first = FIRST(prods[next_symbol_pos])
                    for terms in symbol_first:
                        if terms not in follow:
                            follow.append(terms)
                    if EPSILON in symbol_first:
                        next_symbol_pos +=1
                        follow.remove(EPSILON)
                    else:
                        break

                # CASE 3 - in the last position
                else: # end sym always follows head.
                    follow_head = True

                if follow_head and heads not in follow_seen:
                    for terms in FOLLOW(heads):
                        if terms not in follow:
                            follow.append(terms)

    follow_seen.remove(passed_head)

    return follow

def print_follow() :
    print("\nFOLLOW:")
    for head in G.keys(): # loop through each nonterminals in G
        print(f"{head} = {{",end=' ')
        terms = FOLLOW(head) # find the first of a single head/non-terminals
        print(' , '.join(terms),end=' ')
        print("}")

In [171]:
print_follow()


FOLLOW:
S' = { $ }
S = { $ }
L = { = , $ }
R = { $ , = }


## closure 

In [31]:
def closure(I):
    # J = defaultdict()
    J = I
#     print(J)
#     print(I)
    
    all_heads = J.keys()
    main_heads = list(all_heads)
    
#     item_len = len(J) + sum(len(v) for v in J.values())
#     print(main_heads)

    for heads in main_heads:
#         print(f"for looop {main_heads}")
#         print(heads)
        heads_all_prods = J[heads]
#         print(heads_all_prods)
        for prods in heads_all_prods:
            dot_pos = prods.index('.')      # checks if final item or not
            if dot_pos + 1 < len(prods):
                prod_after_dot = prods[dot_pos + 1]
                if prod_after_dot in nonterminals : # and prod_after_dot not in main_heads: # Non terminals get expansion
                    nonterminals_productions  = G[prod_after_dot]
                    for prod in nonterminals_productions:
                        item = ["."] + prod
                        if prod_after_dot not in main_heads:
                            main_heads.append(prod_after_dot)
                            J[prod_after_dot] = [item]
                        elif item not in J[prod_after_dot]:
                            J[prod_after_dot].append(item)


    return J # A dictionary


In [12]:
# {"S'": [['.', 'S']]}
heads = "L"
temp_prods = ['*', '.', 'R']
x = closure({heads: [temp_prods]})
x

{'L': [['*', '.', 'R']]}

In [178]:
['.']+G[start][0]

['.', 'S']

In [179]:
start_I0 = {start: [['.'] + G[start][0]]}
print(start_I0)
# https://stackoverflow.com/questions/577234/python-extend-for-a-dictionary

{"S'": [['.', 'S']]}


In [ ]:
{
  "S'": [['S']],
  'S' : [['L', '=', 'R'], ['R']],
  'L' : [['*', 'R'], ['id']],
  'R' : [['L']]
 }

In [180]:
start_I0 = {start: [['.'] + G[start][0]]}
# print(start_I0)
# start_I0 = {start: [['.'] + G['L'][1]]}
print(start_I0)
# C = {'I0': closure(start_I0)}
i0 = closure(start_I0)
print(i0)
#print(C)

{"S'": [['.', 'S']]}
{"S'": [['.', 'S']], 'S': [['.', 'L', '=', 'R'], ['.', 'R']], 'L': [['.', '*', 'R'], ['.', 'id']], 'R': [['.', 'L']]}


In [181]:
i0

{"S'": [['.', 'S']],
 'S': [['.', 'L', '=', 'R'], ['.', 'R']],
 'L': [['.', '*', 'R'], ['.', 'id']],
 'R': [['.', 'L']]}

In [182]:
symbols

["S'", 'S', 'L', 'R', '=', '*', 'id']

# ITEM

In [173]:
start_I0 = {start: [['.'] + G[start][0]]}
start_I0


{"S'": [['.', 'S']]}

In [ ]:
["I0",]

In [32]:
C = {} # stores the state, I's
def items():
    global C
    i = 1
    start_I0 = {start: [['.'] + G[start][0]]}
#     print(start_I0)
    C = {'I0': closure(start_I0)}
    C_heads = list(C.keys())
    try:
          for I in C_heads:
#             print(f"----{I}----")
            for X in symbols:
#                 print("----new----")
                if GOTO(C[I], X) and GOTO(C[I], X) not in C.values():
                    item_name = 'I' + str(i)
                    C_heads.append(item_name)
                    C[item_name] = GOTO(C[I], X)
                    i += 1
    except Exception as e:
        print(e)
        pass
    
def print_item() :
    print("\nITEMS:")
    for i in range(len(C)):
        print('I' + str(i) + ':')
        for keys in C['I' + str(i)]:
            for prods in C['I' + str(i)][keys]:
                new_canonical_prods = ' '.join(prods)
                print(f"  {keys} -> {new_canonical_prods}")
        print()

<img src="1g.jpg" alt="Drawing" style="width: 700px;"/>

<img src="2g.jpg" alt="Drawing" style="width: 700px;"/>

In [33]:
def GOTO(I, X):
    goto = {}
#     print(I)
#     print(X)
    for heads in I.keys():
        for prods in I[heads]:
            for i in range(len(prods)-1):
                if "." == prods[i] and X == prods[i + 1]: # select the symbols next to the dot.
                    temp_prods = prods[:]
#                     print(temp_prods)
                    temp_prods[i], temp_prods[i + 1] = temp_prods[i + 1], temp_prods[i]
#                     print(f"heads = {heads}")
#                     print(f"temp_prds = {temp_prods}")
                    prod_closure = closure({heads: [temp_prods]})
#                     print(prod_closure)
                    for keys in prod_closure:
#                         print(keys)
                        if keys not in goto.keys():
                            goto[keys] = prod_closure[keys]
                        elif prod_closure[keys] not in goto[keys]:
                            for prod in prod_closure[keys]:
                                goto[keys].append(prod)
    return goto

In [186]:
def print_goto(J):
    for keys in J:
        for prods in J[keys]:
            new_canonical_prods = ' '.join(prods)
            print(f"{keys} -> {new_canonical_prods}")

In [187]:
I = {"S'": [['.', 'S']], 'S': [['.', 'L', '=', 'R'], ['.', 'R']], 'L': [['.', '*', 'R'], ['.', 'id']], 'R': [['.', 'L']]}
# X = "*"
test_goto = GOTO(I,"*")
print_goto(test_goto)
print("done")

L -> * . R
L -> . * R
L -> . id
R -> . L
done


In [195]:
symbols

["S'", 'S', 'L', 'R', '=', '*', 'id']

In [206]:
start_I0

{"S'": [['.', 'S']],
 'S': [['.', 'L', '=', 'R'], ['.', 'R']],
 'L': [['.', '*', 'R'], ['.', 'id']],
 'R': [['.', 'L']]}

In [194]:
I0 = {
 "S'": [['.', 'S']],
 'S': [['.', 'L', '=', 'R'], ['.', 'R']],
 'L': [['.', '*', 'R'], ['.', 'id']],
 'R': [['.', 'L']]
}

In [211]:
test_goto = GOTO(I0,"*")
print_goto(test_goto)
print("done")

L -> * . R
L -> . * R
L -> . id
R -> . L
done


In [ ]:
{"S'": [['.', 'S']]}

In [ ]:
{'L': [['*', '.', 'R']]}

In [213]:
closure({'L': [['*', '.', 'R']]})

{'L': [['*', '.', 'R'], ['.', '*', 'R'], ['.', 'id']], 'R': [['.', 'L']]}

In [214]:
test_goto

{'L': [['*', '.', 'R'], ['.', '*', 'R'], ['.', 'id']], 'R': [['.', 'L']]}

In [ ]:
I2 = {'S': [['L', '.', '=', 'R']], 'R': [['L', '.']]}
test_goto1 = GOTO(I2,'=')
print_goto(test_goto1)
print("done")

In [ ]:
test_goto1

In [215]:
G = {}
C = {}
start = ""
terminals = []
nonterminals = []
symbols = []
error=0
EPSILON = "/E"

grammars = open("grammar.txt")


parse_grammar(grammars)
# print_grammar()
    
# print_first()
# print_follow()

In [216]:
items()

In [ ]:
print_follow()

In [ ]:
print_table()

In [217]:
print_item()


ITEMS:
I0:
  S' -> . S
  S -> . L = R
  S -> . R
  L -> . * R
  L -> . id
  R -> . L

I1:
  S' -> S .

I2:
  S -> L . = R
  R -> L .

I3:
  S -> R .

I4:
  L -> * . R
  L -> . * R
  L -> . id
  R -> . L

I5:
  L -> id .

I6:
  S -> L = . R
  R -> . L
  L -> . * R
  L -> . id

I7:
  R -> L .

I8:
  L -> * R .

I9:
  S -> L = R .



In [228]:
parse_table

[['', 's4', 's5', '', '1', '2', '3'],
 ['', '', '', 'acc', '', '', ''],
 ['s6/r5', '', '', 'r5', '', '', ''],
 ['', '', '', 'r2', '', '', ''],
 ['', 's4', 's5', '', '', '7', '8'],
 ['r4', '', '', 'r4', '', '', ''],
 ['', 's4', 's5', '', '', '7', '9'],
 ['r5', '', '', 'r5', '', '', ''],
 ['r3', '', '', 'r3', '', '', ''],
 ['', '', '', 'r1', '', '', '']]

In [232]:
print_table()

PARSING TABLE:
+--------+--------+--------+--------+--------+--------+--------+--------+
| STATE  |    =   |    *   |   id   |    $   |    S   |    L   |    R   | 
+--------+--------+--------+--------+--------+--------+--------+--------+
|   0    |        |   s4   |   s5   |        |    1   |    2   |    3   | 
|   1    |        |        |        |   acc  |        |        |        | 
|   2    |  s6/r5 |        |        |   r5   |        |        |        | 
|   3    |        |        |        |   r2   |        |        |        | 
|   4    |        |   s4   |   s5   |        |        |    7   |    8   | 
|   5    |   r4   |        |        |   r4   |        |        |        | 
|   6    |        |   s4   |   s5   |        |        |    7   |    9   | 
|   7    |   r5   |        |        |   r5   |        |        |        | 
|   8    |   r3   |        |        |   r3   |        |        |        | 
|   9    |        |        |        |   r1   |        |        |        | 
+--------+--

In [253]:
ACTION(5,"=")

'r4'

In [254]:
C["I5"]

{'L': [['id', '.']]}

In [280]:
def ACTION(i, a):
    """
        Left for Explaining
        i is state number
        a is symbol
        i -> a , gives what?
    """
    """
    action_test = ACTION(2, 'id') = 's6/r5'
     What if no symbol
     3 Cases :
         1 -> after dot  symbol matches :
                 1.1 -> symbol is terminal      =  reduce
                 1.2 -> symbol is non terminal  =  shift
         3 -> after dot symbol does not match:          
         2 -> dot at end 
    """
    global error
    for heads in C['I' + str(i)]: # loop through each i's Head
        for prods in C['I' + str(i)][heads]: # Loop through production array of the i's head
            for j in range(len(prods) - 1):
                # shift checking  
                if prods[j] == '.' and prods[j + 1] == a: # symbol after dot is "a", okay for goto
                    for k in range(len(C)):
                        if GOTO(C['I' + str(i)], a) == C['I' + str(k)]: # goto of ith state matches to other state (k)
                            if a in terminals: # a is termianl then reduce.
                                if "r" in parse_table[i][terminals.index(a)]: # check r in i,a
                                    if error!=1:
                                        print(f"ERROR: Shift-Reduce Conflict at State {str(i)}, Symbol \' {str(terminals.index(a))} \'")
                                    error=1
                                    if "s"+str(k) not in parse_table[i][terminals.index(a)]:
                                        parse_table[i][terminals.index(a)] = parse_table[i][terminals.index(a)]+ "/s" + str(k)
                                    return parse_table[i][terminals.index(a)]
                                else:
                                    parse_table[i][terminals.index(a)] = "s" + str(k)
                                    return "s" + str(k)
                            else: # Non terminal so shift
                                parse_table[i][len(terminals) + nonterminals.index(a)] = str(k) # i is row, position of a in table 
                            return str(k) #  value of goto state
    
    # after shift check fail 
    # to check if reduce
    for heads in C['I' + str(i)]: # # loop through each i's Head
        if heads != start:
            for prods in C['I' + str(i)][heads]:
                if prods[-1] == '.':             #final item for reduce.
                    k = 0
                    for head in G.keys():
                        for Gprods in G[head]:
                            if head == heads and (Gprods == prods[:-1] ) and (a in terminals or a == '$'):
                                for terms in FOLLOW(heads):
                                    if terms == '$':
                                        index = len(terminals)
                                    else:
                                        index = terminals.index(terms)
                                    if "s" in parse_table[i][index]: # if shift allready exist
                                        if error!=1:
                                            print(f"ERROR: Shift-Reduce Conflict at State {str(i)}, Symbol \' {str(terms)} \'")
                                        error=1
                                        if "r"+str(k) not in parse_table[i][index]:
                                            parse_table[i][index] = parse_table[i][index]+ "/r" + str(k)
                                        return parse_table[i][terminals.index(a)] # return state number,
                                    elif parse_table[i][index] and parse_table[i][index] != "r" + str(k):
                                        if error!=1:
                                            print(f"ERROR: Reduce-Reduce Conflict at State {str(i)}, Symbol \' {str(terms)} \'")
                                        error=1
                                        if "r"+str(k) not in parse_table[i][index]:
                                                parse_table[i][index] = parse_table[i][index]+ "/r" + str(k)
                                        return parse_table[i][terminals.index(a)] # return state number,
                                    else:
                                        parse_table[i][index] = "r" + str(k)
#                                 if         
                                index = terminals.index(a)
                                return parse_table[i][index]
                            k += 1
    if start in C['I' + str(i)] and G[start][0] + ['.'] in C['I' + str(i)][start]: 
        # S' -> S. Case
        parse_table[i][len(terminals)] = "acc"
        return "acc" # Return as accept
#     parse_table[i][len(terminals)] = ""
    return "" # return no action

In [257]:
I0

{"S'": [['.', 'S']],
 'S': [['.', 'L', '=', 'R'], ['.', 'R']],
 'L': [['.', '*', 'R'], ['.', 'id']],
 'R': [['.', 'L']]}

In [ ]:
print_table()

In [296]:
action_test = ACTION(2, "S'")
action_test

''

In [ ]:
parse_table

In [272]:
print_table()

PARSING TABLE:
+--------+--------+--------+--------+--------+--------+--------+--------+
| STATE  |    =   |    *   |   id   |    $   |    S   |    L   |    R   | 
+--------+--------+--------+--------+--------+--------+--------+--------+
|   0    |        |   s4   |   s5   |        |    1   |    2   |    3   | 
|   1    |        |        |        |   acc  |        |        |        | 
|   2    |  s6/r5 |        |        |   r5   |        |        |        | 
|   3    |        |        |        |   r2   |        |        |        | 
|   4    |        |   s4   |   s5   |        |        |    7   |    8   | 
|   5    |   r4   |        |        |   r4   |        |        |        | 
|   6    |        |   s4   |   s5   |        |        |    7   |    9   | 
|   7    |   r5   |        |        |   r5   |        |        |        | 
|   8    |   r3   |        |        |   r3   |        |        |        | 
|   9    |        |        |        |   r1   |        |        |        | 
+--------+--

In [231]:
symbols

["S'", 'S', 'L', 'R', '=', '*', 'id']

In [309]:
def print_table():
    for i in range(len(parse_table)):  #row 9      #len gives number of states
        for j in symbols:              # col
            ACTION(i, j)
            
    print("PARSING TABLE:")
    print("+" + "--------+" * (len(symbols)+ 1))
    print("|{:^8}|".format('STATE'),end=" ")
    
    for terms in terminals:
        print("{:^7}|".format(terms),end=" ")
        
    print("{:^7}|".format("$"),end=" ")
   
    for nonterms in nonterminals:
        if nonterms == start:
            continue
        print("{:^7}|".format(nonterms),end=" ")
    print("\n+" + "--------+" * (len(symbols) + 1))
    
    for i in range(len(parse_table)):
        print("|{:^8}|".format(i),end=' ')
        for j in range(len(symbols)):
            print("{:^7}|".format(parse_table[i][j]),end=' ')
        print()
    print("+" + "--------+" * (len(symbols) + 1))

In [310]:
print_table()

PARSING TABLE:
+--------+--------+--------+--------+--------+--------+--------+--------+
| STATE  |    =   |    *   |   id   |    $   |    S   |    L   |    R   | 
+--------+--------+--------+--------+--------+--------+--------+--------+
|   0    |        |   s4   |   s5   |        |    1   |    2   |    3   | 
|   1    |        |        |        |   acc  |        |        |        | 
|   2    |  s6/r5 |        |        |   r5   |        |        |        | 
|   3    |        |        |        |   r2   |        |        |        | 
|   4    |        |   s4   |   s5   |        |        |    7   |    8   | 
|   5    |   r4   |        |        |   r4   |        |        |        | 
|   6    |        |   s4   |   s5   |        |        |    7   |    9   | 
|   7    |   r5   |        |        |   r5   |        |        |        | 
|   8    |   r3   |        |        |   r3   |        |        |        | 
|   9    |        |        |        |   r1   |        |        |        | 
+--------+--

In [307]:
global parse_table

no_of_rows = len(C) # No of States, right side of table
no_of_cols = len(symbols)  # All symbols plus the state no
parse_table = [["" for c in range(no_of_cols) ] for r in range(no_of_rows)]
# print_table()

In [308]:
parse_table

[['', '', '', '', '', '', ''],
 ['', '', '', '', '', '', ''],
 ['', '', '', '', '', '', ''],
 ['', '', '', '', '', '', ''],
 ['', '', '', '', '', '', ''],
 ['', '', '', '', '', '', ''],
 ['', '', '', '', '', '', ''],
 ['', '', '', '', '', '', ''],
 ['', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '']]

In [39]:
G

{"S'": [['S']],
 'S': [['L', '=', 'R'], ['R']],
 'L': [['*', 'R'], ['id']],
 'R': [['L']]}

In [38]:
G = {} # Augmented Grammar 
C = {} # I haru bascha
start = "" # S'
terminals = [] # 
nonterminals = [] #
symbols = [] # terminals + non terminals
error=0 # 
EPSILON = "/E" # 

grammars = open("grammar.txt")


parse_grammar(grammars)
print_grammar()
    
print_first()
print_follow()

items()
print_item()

global parse_table
no_of_rows = len(C) # No of States, right side of table
no_of_cols = len(symbols)  # All symbols plus the state no - S'
# no_of_cols
# parse_table = [["" for c in range(len(terminals) + len(nonterminals) + 1)] for r in range(len(C))]
parse_table = [["" for c in range(no_of_cols) ] for r in range(no_of_rows)]

print_table()

# process_input()


GRAMMAR:
 S -> L = R | R 
 L -> * R | id 
 R -> L 

AUGMENTED GRAMMAR:
0: S' -> S
1: S -> L = R
2: S -> R
3: L -> * R
4: L -> id
5: R -> L

TERMINALS    : ['=', '*', 'id']
NONTERMINALS : ["S'", 'S', 'L', 'R']
SYMBOLS      : ["S'", 'S', 'L', 'R', '=', '*', 'id']

FIRST:
S' = { * , id }
S = { * , id }
L = { * , id }
R = { * , id }

FOLLOW:
S' = { $ }
S = { $ }
L = { = , $ }
R = { $ , = }

ITEMS:
I0:
  S' -> . S
  S -> . L = R
  S -> . R
  L -> . * R
  L -> . id
  R -> . L

I1:
  S' -> S .

I2:
  S -> L . = R
  R -> L .

I3:
  S -> R .

I4:
  L -> * . R
  L -> . * R
  L -> . id
  R -> . L

I5:
  L -> id .

I6:
  S -> L = . R
  R -> . L
  L -> . * R
  L -> . id

I7:
  R -> L .

I8:
  L -> * R .

I9:
  S -> L = R .

ERROR: Shift-Reduce Conflict at State 2, Symbol ' = '
PARSING TABLE:
+--------+--------+--------+--------+--------+--------+--------+--------+
| STATE  |    =   |    *   |   id   |    $   |    S   |    L   |    R   | 
+--------+--------+--------+--------+--------+--------+------

<img src="3g.jpg" alt="Drawing" style="width: 700px;"/>

In [ ]:
print_grammar()

In [311]:
print_table()

PARSING TABLE:
+--------+--------+--------+--------+--------+--------+--------+--------+
| STATE  |    =   |    *   |   id   |    $   |    S   |    L   |    R   | 
+--------+--------+--------+--------+--------+--------+--------+--------+
|   0    |        |   s4   |   s5   |        |    1   |    2   |    3   | 
|   1    |        |        |        |   acc  |        |        |        | 
|   2    |  s6/r5 |        |        |   r5   |        |        |        | 
|   3    |        |        |        |   r2   |        |        |        | 
|   4    |        |   s4   |   s5   |        |        |    7   |    8   | 
|   5    |   r4   |        |        |   r4   |        |        |        | 
|   6    |        |   s4   |   s5   |        |        |    7   |    9   | 
|   7    |   r5   |        |        |   r5   |        |        |        | 
|   8    |   r3   |        |        |   r3   |        |        |        | 
|   9    |        |        |        |   r1   |        |        |        | 
+--------+--

In [349]:
def look_at_parse_table(top_stack,curr_symbol):
    """
        top_stack  -> no. of state
    """
    if curr_symbol=="$":
        start_index  = len(terminals) # terminal 
        return parse_table[top_stack][start_index]
    elif curr_symbol in terminals:
        term_index = terminals.index(curr_symbol)
        return parse_table[top_stack][term_index]
    elif curr_symbol in nonterminals:
        nonterm_index = len(terminals) + nonterminals.index(curr_symbol)
#         print(nonterm_index)
        return parse_table[top_stack][nonterm_index]
    else:
        return "none"

In [ ]:
look_at_parse_table(0,"R")

In [ ]:
*,id,$

In [328]:
isn = "   * id    " 

In [329]:
isn + " $"

'   * id     $'

In [330]:
(isn + " $").split()

['*', 'id', '$']

In [326]:
" ".join((isn + " $").split())

'* id $'

In [327]:
" ".join((isn + " $").split()).split(" ")

['*', 'id', '$']

In [369]:
isn

'   * id    '

In [368]:
to_parse = (isn + " $").split()
to_parse

['*', 'id', '$']

In [381]:
process_input(isn)


+--------+----------------------------+----------------------------+----------------------------+
|  STEP  |           STACK            |           INPUT            |           ACTION           | +--------+----------------------------+----------------------------+----------------------------+
|   1    | 0                          |                       *id$ |              s4            |
|   2    | 0*4                        |                        id$ |              s5            |
|   3    | 0*4id5                     |                          $ |              r4            |
|   4    | 0*4L7                      |                          $ |              r5            |
|   5    | 0*4R8                      |                          $ |              r3            |
|   6    | 0L2                        |                          $ |              r5            |
|   7    | 0R3                        |                          $ |              r2            |
|   8    | 0S1     

In [387]:
print_table()

PARSING TABLE:
+--------+--------+--------+--------+--------+--------+--------+--------+
| STATE  |    =   |    *   |   id   |    $   |    S   |    L   |    R   | 
+--------+--------+--------+--------+--------+--------+--------+--------+
|   0    |        |   s4   |   s5   |        |    1   |    2   |    3   | 
|   1    |        |        |        |   acc  |        |        |        | 
|   2    |  s6/r5 |        |        |   r5   |        |        |        | 
|   3    |        |        |        |   r2   |        |        |        | 
|   4    |        |   s4   |   s5   |        |        |    7   |    8   | 
|   5    |   r4   |        |        |   r4   |        |        |        | 
|   6    |        |   s4   |   s5   |        |        |    7   |    9   | 
|   7    |   r5   |        |        |   r5   |        |        |        | 
|   8    |   r3   |        |        |   r3   |        |        |        | 
|   9    |        |        |        |   r1   |        |        |        | 
+--------+--

In [386]:
def process_input(get_input):
#     get_input = input("\nEnter Input: ")
    to_parse = (get_input + " $").split()
#     * id
    curr_pointer = 0  # Current input symbol pointer
    stack = ['0']

    print("\n+--------+----------------------------+----------------------------+----------------------------+")
    print("|{:^8}|{:^28}|{:^28}|{:^28}|".format("STEP", "STACK", "INPUT", "ACTION"))
    print("+--------+----------------------------+----------------------------+----------------------------+")

    step = 1
    while True:
        curr_symbol = to_parse[curr_pointer]
        top_stack = int(stack[-1])
#         stack_content = ""
        input_content = ""

        print("|{:^8}|".format(step),end=" ")
        
        stack_content = "".join(stack)
        print("{:27}|".format(stack_content),end=' ')
#         input_content = "".join(to_parse)
        i = curr_pointer
        while i < len(to_parse):
            input_content += to_parse[i]
            i += 1
        print("{:>26} | ".format(input_content),end=' ')

        step += 1
        
        get_action = look_at_parse_table(top_stack,curr_symbol)
        
#         get_action = ACTION(top_stack, curr_symbol)
        if "/" in get_action:
            print("{:^26}|".format(get_action+". So conflict"))
            break
        if "s" in get_action:
            print("{:^26}|".format(get_action))
            stack.append(curr_symbol)
            stack.append(get_action[1:])
            curr_pointer += 1
        elif "r" in get_action:
            print("{:^26}|".format(get_action))
            i = 0
            for head in G.keys():
                for prods in G[head]:
                    if i == int(get_action[1:]):
                        for j in range(2 * len(prods)):
                            stack.pop()
                        state = stack[-1]
                        stack.append(head)
                        stack.append(parse_table[int(state)][len(terminals) + nonterminals.index(head)])
                    i += 1
        elif get_action == "acc":
            print("{:^26}|".format("ACCEPTED"))
            break
        elif get_action == "" :
#             print(f"ERROR: Unrecognized symbol {curr_symbol} |")
            print("{:^26}|".format("Not Accepted "))
            break
        else:
            print(f"Error : Invalid symbol {curr_symbol}  |")
#             print("{:^26}|".format("Not Accepted "))
#             print(f"ERROR: Unrecognized symbol {curr_symbol} |")
            break
    print("+--------+----------------------------+----------------------------+----------------------------+")

In [ ]:
print_grammar()

In [223]:
parse_table

[['', 's4', 's5', '', '1', '2', '3', ''],
 ['', '', '', 'acc', '', '', '', ''],
 ['s6/r5', '', '', 'r5', '', '', '', ''],
 ['', '', '', 'r2', '', '', '', ''],
 ['', 's4', 's5', '', '', '7', '8', ''],
 ['r4', '', '', 'r4', '', '', '', ''],
 ['', 's4', 's5', '', '', '7', '9', ''],
 ['r5', '', '', 'r5', '', '', '', ''],
 ['r3', '', '', 'r3', '', '', '', ''],
 ['', '', '', 'r1', '', '', '', '']]

In [315]:
print_table()

PARSING TABLE:
+--------+--------+--------+--------+--------+--------+--------+--------+
| STATE  |    =   |    *   |   id   |    $   |    S   |    L   |    R   | 
+--------+--------+--------+--------+--------+--------+--------+--------+
|   0    |        |   s4   |   s5   |        |    1   |    2   |    3   | 
|   1    |        |        |        |   acc  |        |        |        | 
|   2    |  s6/r5 |        |        |   r5   |        |        |        | 
|   3    |        |        |        |   r2   |        |        |        | 
|   4    |        |   s4   |   s5   |        |        |    7   |    8   | 
|   5    |   r4   |        |        |   r4   |        |        |        | 
|   6    |        |   s4   |   s5   |        |        |    7   |    9   | 
|   7    |   r5   |        |        |   r5   |        |        |        | 
|   8    |   r3   |        |        |   r3   |        |        |        | 
|   9    |        |        |        |   r1   |        |        |        | 
+--------+--

In [362]:
get_input = "id * id   "
process_input(get_input)


+--------+----------------------------+----------------------------+----------------------------+
|  STEP  |           STACK            |           INPUT            |           ACTION           | +--------+----------------------------+----------------------------+----------------------------+
|   1    | 0                          |                     id*id$ |              s5            |
|   2    | 0id5                       |                       *id$ |        Not Accepted        |
+--------+----------------------------+----------------------------+----------------------------+


GRAMMAR:
 S -> L = R | R 
 L -> * R | id 
 R -> L 

AUGMENTED GRAMMAR:
0: S' -> S
1: S -> L = R
2: S -> R
3: L -> * R
4: L -> id
5: R -> L

TERMINALS    : ['=', '*', 'id']
NONTERMINALS : ["S'", 'S', 'L', 'R']
SYMBOLS      : ["S'", 'S', 'L', 'R', '=', '*', 'id']

FIRST:
S' = { * , id }
S = { * , id }
L = { * , id }
R = { * , id }

FOLLOW:
S' = { $ }
S = { $ }
L = { = , $ }
R = { $ , = }

ITEMS:
I0:
  S' -> . S
  S -> . L = R
  S -> . R
  L -> . * R
  L -> . id
  R -> . L

I1:
  S' -> S .

I2:
  S -> L . = R
  R -> L .

I3:
  S -> R .

I4:
  L -> * . R
  L -> . * R
  L -> . id
  R -> . L

I5:
  L -> id .

I6:
  S -> L = . R
  R -> . L
  L -> . * R
  L -> . id

I7:
  R -> L .

I8:
  L -> * R .

I9:
  S -> L = R .

ERROR: Shift-Reduce Conflict at State 2, Symbol ' = '
PARSING TABLE:
+--------+--------+--------+--------+--------+--------+--------+--------+
| STATE  |    =   |    *   |   id   |    $   |    S   |    L   |    R   | 
+--------+--------+--------+--------+--------+--------+------


Enter Input:  * id



+--------+----------------------------+----------------------------+----------------------------+
|  STEP  |           STACK            |           INPUT            |           ACTION           |
+--------+----------------------------+----------------------------+----------------------------+
|   1    | 0                          |                       *id$ |              s4            |
|   2    | 0*4                        |                        id$ |              s5            |
|   3    | 0*4id5                     |                          $ |              r4            |
|   4    | 0*4L7                      |                          $ |              r5            |
|   5    | 0*4R8                      |                          $ |              r3            |
|   6    | 0L2                        |                          $ |              r5            |
|   7    | 0R3                        |                          $ |              r2            |
|   8    | 0S1     

In [398]:
I = {}
J = {}
relation = []
r1 = []

def construct_dfa():
    Z = []
    pd = []
#     print("\nITEMS:")
    for i in range(len(C)):
#         print(f"I{str(i)} :")
        I[i] = 'I' + str(i)
        Z = ""
        for keys in C['I' + str(i)]: # keys = head
            Y = ""
            for prods in C['I' + str(i)][keys]:
                # print(G)
#                 zzz = "{:>{width}} ->".format(keys, width=len(max(G.keys(), key=len)))
                zzz = f" {keys} ->"
                pd = ""

                Z = Z + zzz
#                 print(zzz,end=' ')
                pd = ''.join(prods)
#                 print(pd)
                
                Z = Z + pd + "\n"
#                 print(f"z {Z}")
#                 print()
            Y = Y + Z
#         print()
#         print("J")
        J[i] = Y
#         print(Y,end=' ')
        
    for i in range(len(parse_table)):
        for j in symbols:
            ACTION(i, j)
    global dot

    dot = Digraph()
    
    for i in range(len(C)):
        for a in symbols:
            rel = look_at_parse_table(i,a)
            if rel:
                # print rel
                if (len(rel) == 1):
                    r = int(rel)
                else:
                    if (rel == 'acc') or (rel[0] == 'r'):
                        continue
                    elif '/' in rel:
                        spos = rel.index('s')
                        rel = rel[spos:spos + 2]
#                         print(rel)
                        r = int(rel[1:3])
                    else:
                        # print rel
                        r = int(rel[1:3])

#                 print("node %d relates to %s for %s" % (i, r, a))
                relation.append(chr(i + 97) + chr(r + 97))
                r1.append(a)

#     print(relation,end=' ')
#     print(r1,end=' ')

    M = [v for v in I.values()]
    N = [v for v in J.values()]

    for i in range(len(C)):
        dot.node(chr(97 + i), N[i], xlabel=M[i])

    for i in range(len(relation)):
        dot.edge(relation[i][0], relation[i][1], label=r1[i])
#     dot.render('./test.gv.svg', view=True)
    dot.render(filename='g1.dot')

In [399]:
construct_dfa()

In [ ]:
%ls

In [400]:
%dot

UsageError: Line magic function `%dot` not found.


In [ ]:
%cat grammar.txt

In [394]:
%dot -Tpng g1.dot -o okay2.png 

UsageError: Line magic function `%dot` not found.


<img src="ok.png" alt="Drawing" style="width: 700px;"/>